In [ ]:
import csv
import pandas as pd
import os

In [ ]:
directory = 'pkl'

for filename in os.listdir(directory):
    if filename.endswith('.pkl') and "uber-raw-data" in filename:
        file_path = os.path.join(directory, filename)

        df = pd.read_pickle(file_path)
        
        print(f"File: {filename}")
        print(f"Columns: {df.columns.tolist()}\n")
        try:
            print(f"Max Lon: {df['Lon'].max()}")
            print(f"Max Lon: {df['Lon'].min()}")
            print(f"Max Lat: {df['Lat'].max()}")
            print(f"Max Lat: {df['Lat'].min()}")
        except Exception as e:
            print("Does not have column Lon or Lat")